<a href="https://colab.research.google.com/github/pancongkeju/sosial-media-analyst/blob/main/Collecting_Data_From_Youtube_Using_Keyword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

1. Download List Video sesuai keyword

In [ ]:
import os
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import csv

# Jika dijalankan di Google Colab, mount Google Drive:
try:
    from google.colab import drive
    drive.mount('')
    drive_path = ''  # ubah sesuai folder target Anda
except ImportError:
    # Jika bukan Colab, simpan di local folder
    drive_path = os.getcwd()

# -- KONFIGURASI API KEY --
API_KEY = ""

# Bangun client YouTube
youtube = build("youtube", "v3", developerKey=API_KEY)

# Parameter pencarian
QUERY = "Mega Thrust Indonesia"
REGION = "ID"  # kode negara Indonesia
MAX_RESULTS = 50  # batas maksimum per request

# Fungsi bantu untuk dapatkan tanggal mulai bulan berikutnya
def next_month(dt: datetime) -> datetime:
    if dt.month == 12:
        return datetime(dt.year + 1, 1, 1)
    else:
        return datetime(dt.year, dt.month + 1, 1)

# Rentang total: 1 Jan 2024 sampai 30 Apr 2025
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 4, 30, 23, 59, 59)

results = []

current_after = start_date
while current_after <= end_date:
    current_before = next_month(current_after) - timedelta(seconds=1)
    if current_before > end_date:
        current_before = end_date

    publishedAfter = current_after.strftime("%Y-%m-%dT%H:%M:%SZ")
    publishedBefore = current_before.strftime("%Y-%m-%dT%H:%M:%SZ")

    next_page_token = None
    while True:
        # Ambil satu halaman hasil pencarian, diurutkan berdasarkan tanggal
        search_response = youtube.search().list(
            part="snippet",
            q=QUERY,
            type="video",
            regionCode=REGION,
            publishedAfter=publishedAfter,
            publishedBefore=publishedBefore,
            order="date",
            maxResults=MAX_RESULTS,
            pageToken=next_page_token
        ).execute()

        # Kumpulkan ID video untuk batch request statistik
        video_ids = [item["id"]["videoId"] for item in search_response.get("items", [])]
        if not video_ids:
            break

        stats_response = youtube.videos().list(
            part="statistics",
            id=",".join(video_ids)
        ).execute()

        # Mapping videoId → commentCount
        comment_counts = {
            item["id"]: int(item["statistics"].get("commentCount", 0))
            for item in stats_response.get("items", [])
        }

        # Tambahkan setiap video ke list hasil
        for snippet in search_response.get("items", []):
            vid = snippet["id"]["videoId"]
            results.append({
                "channel": snippet["snippet"]["channelTitle"],
                "title": snippet["snippet"]["title"],
                "url": f"https://www.youtube.com/watch?v={vid}",
                "publishedAt": snippet["snippet"]["publishedAt"][:10],
                "comments": comment_counts.get(vid, 0)
            })

        next_page_token = search_response.get("nextPageToken")
        if not next_page_token:
            break

    current_after = next_month(current_after)

# Simpan hasil ke CSV di Google Drive (atau local jika tidak di Colab)
output_file = os.path.join(drive_path, "youtube_results.csv")
with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["channel", "title", "url", "publishedAt", "comments"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for vid in results:
        writer.writerow(vid)

print(f"Hasil pencarian disimpan di {output_file}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Hasil pencarian disimpan di /content/drive/MyDrive/megathrust4/youtube_results.csv


In [ ]:
import pandas as pd

# 1. Baca file CSV (sesuaikan path jika perlu)
# input_csv = '/content/drive/MyDrive/
input_csv = '/content/drive/Myoutube_results_edit.csv'

df = pd.read_csv(input_csv)

# 2. Daftar channel yang ingin dipertahankan
channels = [ "CNN Indonesia", 'METRO TV',]

# 3. Filter: comments > 0
df = df[df['comments'] > 0]

# 4. Filter: hanya channel di dalam daftar
df = df[df['channel'].isin(channels)]

# 5. Filter: hanya channel yg tervalidasi
df = df[df['keterangan'] == 1]

# (Opsional) Simpan hasil ke CSV baru
output_csv = '/content/drive/MyDrive/filtered_megathrust_youtube_results.csv'
df.to_csv(output_csv, index=False)

print(f"Hasil filter disimpan di {output_csv}")

Hasil filter disimpan di /content/drive/MyDrive/megathrust4/filtered_megathrust_youtube_results.csv


In [ ]:
# Jika pakai Colab: mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import time
import csv
import pandas as pd
from urllib.parse import urlparse, parse_qs
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

API_KEY = ""

# 2. Path ke CSV hasil filter
input_csv = "/content/drive/MyDrive/filtered_megathrust_youtube_results.csv"
df = pd.read_csv(input_csv)

# 3. Fungsi ekstrak video_id dari URL
def extract_video_id(url):
    parsed = urlparse(url)
    if "youtu.be" in parsed.netloc:
        return parsed.path.lstrip("/")
    qs = parse_qs(parsed.query)
    return qs.get("v", [None])[0]

# 4. Siapkan file output dan tulis header
output_csv = "/content/drive/MyDrive/all_comments.csv"
with open(output_csv, "w", newline="", encoding="utf-8") as fout:
    writer = csv.writer(fout)
    writer.writerow(["url_video", "namachannel", "nama_user", "komen", "tanggal_komen"])

    # 5. Loop setiap video
    for _, row in df.iterrows():
        url = row["url"]
        channel = row["channel"]
        vid = extract_video_id(url)
        if not vid:
            print(f"[SKIP] Gagal ekstrak video ID dari {url}")
            continue

        print(f"[FETCH] Comments for {vid}")
        page_token = None
        backoff = 1

        # 6. Paginate untuk seluruh top-level comments + replies
        while True:
            try:
                req = youtube.commentThreads().list(
                    part="snippet,replies",
                    videoId=vid,
                    maxResults=100,
                    textFormat="plainText",
                    pageToken=page_token
                )
                res = req.execute()

                # 7. Tulis top-level comment
                for item in res.get("items", []):
                    top = item["snippet"]["topLevelComment"]["snippet"]
                    writer.writerow([
                        url,
                        channel,
                        top.get("authorDisplayName", ""),
                        top.get("textDisplay", "").replace("\n", " "),
                        top.get("publishedAt", "")
                    ])
                    # 8. Tulis replies (jika ada)
                    for reply in item.get("replies", {}).get("comments", []):
                        r = reply["snippet"]
                        writer.writerow([
                            url,
                            channel,
                            r.get("authorDisplayName", ""),
                            r.get("textDisplay", "").replace("\n", " "),
                            r.get("publishedAt", "")
                        ])

                page_token = res.get("nextPageToken")
                backoff = 1  # reset backoff jika sukses
                if not page_token:
                    break

                time.sleep(1)  # throttle antar halaman

            except HttpError as e:
                status = e.resp.status
                if status in [403, 500, 503]:
                    # quota atau server error: exponential backoff
                    print(f"[WARN] HTTP {status}, backoff {backoff}s")
                    time.sleep(backoff)
                    backoff = min(backoff * 2, 60)
                else:
                    print(f"[ERROR] HttpError {status} pada video {vid}: {e}")
                    break

print(f"Selesai! Semua komentar tersimpan di: {output_csv}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[FETCH] Comments for ENrp8zglxOI
[FETCH] Comments for D_e2oRugqo8
[FETCH] Comments for ReS0FIGo4go
[FETCH] Comments for pM8BaoFPGwE
[FETCH] Comments for jqnH6UZbJdg
[FETCH] Comments for C9_z-QvO0EE
[FETCH] Comments for TSj0xnn8vsc
[FETCH] Comments for ufSjYoY7370
[FETCH] Comments for UOkzlNwZSBM
[FETCH] Comments for yPzE55ZdUJA
[FETCH] Comments for WxGcURUVZz4
[FETCH] Comments for dysgONJYxOk
[FETCH] Comments for 6kXuwnSn0qg
[FETCH] Comments for i2E6GaMitaY
[FETCH] Comments for j4f4arSS_oU
[FETCH] Comments for nOvXWyICm4Y
Selesai! Semua komentar tersimpan di: /content/drive/MyDrive/megathrust4/all_comments.csv
